In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import json
from haversine import haversine

print(datetime.datetime.now())
print(os.getcwd())

# Using the Eexperian

2020-04-01 13:42:37.756891
/home/jian/Projects/Big_Lots/Analysis/2020_Q1/plannder_requests/JT_zips_income_crossvalidation_20200331


In [2]:
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

In [3]:
previous_excel_file=pd.ExcelFile("./BL_zip_cd_income_validation_JL_2020-03-31.xlsx")
previous_excel_file.sheet_names

['validate_1_BL_Exp_data', 'validate_1_JP_Zip4_data']

In [4]:
df_1_exp=previous_excel_file.parse("validate_1_BL_Exp_data")
df_1_exp['customer_zip_code']=df_1_exp['customer_zip_code'].apply(lambda x: str(x).zfill(5))
df_2_EASI=previous_excel_file.parse("validate_1_JP_Zip4_data",dtype=str)
valid_zip_list=df_2_EASI['ZIP_CODE'].tolist()
df_1_exp=df_1_exp[df_1_exp['customer_zip_code'].isin(valid_zip_list)]

In [5]:
df_store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200301-134228-900.txt",
                           dtype=str,sep="|",usecols=['location_id','zip_cd','latitude_meas','longitude_meas'])
df_store_list['zip_cd']=df_store_list['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
df_store_list['latitude_meas']=df_store_list['latitude_meas'].astype(float)
df_store_list['longitude_meas']=df_store_list['longitude_meas'].astype(float)
df_store_list=df_store_list[~df_store_list['location_id'].isin(['145','6990'])]
df_store_list=df_store_list[['location_id','zip_cd','latitude_meas','longitude_meas']]
df_store_list=df_store_list.rename(columns={"zip_cd":"store_zip"})
df_store_list.head(2)

,location_id,store_zip,latitude_meas,longitude_meas
0,3,30906,33.421570,-82.018921
1,30,43402,41.363132,-83.650909


In [6]:
TA_zips=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
TA_zips=TA_zips.parse("view_by_store",dtype=str)
TA_zips.head(2)

,TA_num,ta_name,location_id,address_line_1,address_line_2,zip_cd,city_nm,state_nm,latitude_meas,longitude_meas,DMA,CTY,all_trans_P_zips,all_trans_S_zips,zips_in_10,trans_P_zips_70_within_TA,trans_S_zips_70_within_TA
0,1,multiple_1,1949,3178 LAVON DR,NaN,75040,GARLAND,TX,32.945015,-96.619194,{'DALLAS-FT. WORTH'},{'DALLAS'},"['75040', '75044', '75048', '75098']","['75089', '75042', '75043', '75041', '75088', ...",[],"['75040', '75098', '75044', '75048']","['75088', '75042', '75082', '75043', '75089', ..."
1,1,multiple_1,1255,1418 W MOORE AVE,NaN,75160,TERRELL,TX,32.738772,-96.299877,{'DALLAS-FT. WORTH'},"{'HUNT', 'KAUFMAN', 'ROCKWALL'}","['75160', '75126', '75161']","['75142', '75169', '75474', '75143', '75114', ...",[],"['75126', '75161', '75160']","['75142', '75169', '75150', '75117', '75143', ..."


In [7]:
df_temporary=TA_zips[['location_id','trans_P_zips_70_within_TA','trans_S_zips_70_within_TA','zips_in_10']]
df_zip_by_store=pd.DataFrame()

for ind,row in df_temporary.iterrows():
    location_id=row['location_id']
    P_zips=eval(row['trans_P_zips_70_within_TA'])
    S_zips=eval(row['trans_S_zips_70_within_TA'])
    zip_10=eval(row['zips_in_10'])
    

    df_P=pd.DataFrame(zip([location_id]*len(P_zips),P_zips))
    if len(df_P)>0:
        df_P.columns=['location_id','zip_cd']
        df_P['zip_type']="P"
        
    df_S=pd.DataFrame(zip([location_id]*len(S_zips),S_zips))
    if len(df_S)>0:
        df_S.columns=['location_id','zip_cd']
        df_S['zip_type']="S"
    
    df_10=pd.DataFrame(zip([location_id]*len(zip_10),zip_10))
    if len(df_10)>0:
        df_10.columns=['location_id','zip_cd']
        df_10['zip_type']="zip_10"
    
    df_zip_by_store=df_zip_by_store.append(df_P).append(df_S).append(df_10)

In [8]:
df_output=pd.merge(df_store_list,df_zip_by_store,on="location_id",how="left")
df_output.head(2)

,location_id,store_zip,latitude_meas,longitude_meas,zip_cd,zip_type
0,3,30906,33.42157,-82.018921,30906,P
1,3,30906,33.42157,-82.018921,30904,S


In [9]:
df_output_2=df_output[pd.isnull(df_output['zip_cd'])]
df_output_1=df_output[pd.notnull(df_output['zip_cd'])]
print(df_output_2.shape)

del df_output_2['zip_cd']
del df_output_2['zip_type']

(31, 6)


In [10]:
df_10_zips_all_new_stores=pd.DataFrame()
for ind, row in df_output_2.iterrows():
    location_id=row['location_id']
    store_location=(row['latitude_meas'],row['longitude_meas'])
    for zip_cd,v in zip_centers.items():
        dist=haversine(store_location,v,unit="mi")
        if dist<=10:
            # print(location_id,zip_cd)
            df=pd.DataFrame({"location_id":[location_id],"zip_cd":[zip_cd],"dist":[dist]})
            df_10_zips_all_new_stores=df_10_zips_all_new_stores.append(df)
df_10_zips_all_new_stores['zip_type']="zip_10"
print(df_10_zips_all_new_stores['location_id'].nunique())

31


In [11]:
df_new_store_zips=df_10_zips_all_new_stores[['location_id','zip_cd','zip_type']]
df_output_2=pd.merge(df_output_2,df_new_store_zips,how="left")
df_output=df_output_1.append(df_output_2)
print(df_output.shape,df_output['location_id'].nunique(),df_output['zip_cd'].nunique())

(23350, 6) 1403 13237


In [12]:
df_unique_zip_label=df_output[['zip_cd','zip_type']].sort_values(['zip_cd','zip_type'])
df_unique_zip_label['zip_type']=df_unique_zip_label['zip_type'].replace("zip_10","P")
df_unique_zip_label=df_unique_zip_label.sort_values(['zip_cd','zip_type'])
df_unique_zip_label=df_unique_zip_label.drop_duplicates("zip_cd")
df_unique_zip_label=df_unique_zip_label.rename(columns={"zip_cd":"customer_zip_code"})
df_unique_zip_label.shape

(13237, 2)

In [13]:
df_1_exp=pd.merge(df_1_exp,df_unique_zip_label,on="customer_zip_code",how="left")

In [15]:
df_1_exp['id_per_zip'].sum()

15590547

In [16]:
writer=pd.ExcelWriter("./BL_zip_list_based_on_17mm_Experian_data_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_1_exp.to_excel(writer,"zips_with_TA_label",index=False)
writer.save()